# Домашнее задание №2

**Максимальная оценка за работу `min(X, 10)`, где X - суммарный балл за hw2 из 12 возможных.**

## Часть 1. ML workflow (**всего 5 баллов**)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы.

In [2]:
df = pd.read_csv("/Users/kh11vika/Downloads/winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [64]:
y = df.iloc[:, -1] 
X = df.iloc[:, :-1]

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=16)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [66]:
reg = LinearRegression().fit(X_train, y_train)
y_pred_mse = reg.predict(X_test)
y_pred_train_mse = reg.predict(X_train)

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [67]:
y_pred_train = reg.predict(X_train)
y_pred_test = reg.predict(X_test)

print(f"MSE на тренировочных данных: {mean_squared_error(y_train, y_pred_train):}")
print(f"MSE на тестовых данных: {mean_squared_error(y_test, y_pred_test):}")
print(f"R² на тренировочных данных: {r2_score(y_train, y_pred_train):}")
print(f"R² на тестовых данных: {r2_score(y_test, y_pred_test):}")

MSE на тренировочных данных: 0.4029193184207987
MSE на тестовых данных: 0.4641292014818482
R² на тренировочных данных: 0.38369769677763454
R² на тестовых данных: 0.28169107467930066


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [68]:
r2_scores = cross_val_score(reg, X, y, cv=5, scoring='r2')

print(f"Среднее R² на кросс-валидации с 5 фолдами: {r2_scores.mean():.4f}")

Среднее R² на кросс-валидации с 5 фолдами: 0.2900


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1.

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [69]:
from sklearn.linear_model import Lasso

best_alpha = ''
best_r2 = float('inf')  
results = []

for alpha in np.arange(0.1, 1.1, 0.1):
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)

    y_pred_tr = lasso.predict(X_train)

    train_mse = mean_squared_error(y_train, y_pred_tr)
    train_r2 = r2_score(y_train, y_pred_tr)

    results.append((alpha, train_mse, train_r2)) 

    print(f'alpha={alpha:.1f}, Train MSE: {train_mse:.4f}, Train R²: {train_r2:.4f}')

alpha=0.1, Train MSE: 0.4774, Train R²: 0.2698
alpha=0.2, Train MSE: 0.5185, Train R²: 0.2070
alpha=0.3, Train MSE: 0.5648, Train R²: 0.1361
alpha=0.4, Train MSE: 0.6295, Train R²: 0.0371
alpha=0.5, Train MSE: 0.6316, Train R²: 0.0339
alpha=0.6, Train MSE: 0.6317, Train R²: 0.0338
alpha=0.7, Train MSE: 0.6318, Train R²: 0.0336
alpha=0.8, Train MSE: 0.6319, Train R²: 0.0334
alpha=0.9, Train MSE: 0.6321, Train R²: 0.0332
alpha=1.0, Train MSE: 0.6323, Train R²: 0.0329


### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [70]:
alphas = np.arange(0.1, 1.1, 0.1) 
params = {'alpha': alphas}

cv = GridSearchCV(lasso, params, scoring='r2', cv=5)

cv.fit(X_train, y_train)

print('Best estimator :', cv.best_estimator_)
print('Best score (R²):', cv.best_score_)

Best estimator : Lasso(alpha=0.1)
Best score (R²): 0.26162552533012484


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [15]:
y_pred_test = cv.best_estimator_.predict(X_test)
test_r2_score = r2_score(y_test, y_pred_test)

print(f'R² на тестовых данных: {test_r2_score:.4f}')

R² на тестовых данных: 0.1886


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [71]:
from sklearn.preprocessing import StandardScaler

linear_pipe = Pipeline([('poly', PolynomialFeatures(degree=2)),
                       ('linear_model', LinearRegression())])
print('R²:', cross_val_score(linear_pipe, X, y, cv=5, scoring='r2').mean())

R²: 0.23009616946201242


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [12]:
linear_pipe.fit(X_train, y_train)

y_pred_train_pipe = linear_pipe.predict(X_train)
y_pred_test_pipe = linear_pipe.predict(X_test)

print(f"MSE на тренировочных данных: {mean_squared_error(y_train, y_pred_train_pipe):.4f}")
print(f"MSE на тестовых данных: {mean_squared_error(y_test, y_pred_test_pipe):.4f}")
print(f"R² на тренировочных данных: {r2_score(y_train, y_pred_train_pipe):.4f}")
print(f"R² на тестовых данных: {r2_score(y_test, y_pred_test_pipe):.4f}")

MSE на тренировочных данных: 1.4545
MSE на тестовых данных: 1.7488
R² на тренировочных данных: -1.2248
R² на тестовых данных: -1.7065


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

#### 1. Оценка линейной модели без регуляризации
Линейная модель без регуляризации показала себя не очень хорошо. Значение R² меньше 0.5 говорит о том, что модель не справляется с задачей. Однако трудно точно сказать, есть ли переобучение. Мы видим лишь небольшое ухудшение метрики на тестовой выборке (около 0.1) по сравнению с обучающей. Различия между ошибками и значениями R² тоже невелики, так что делать однозначные выводы о переобучении сложно. К тому же, если поменять параметр random_state, результаты немного меняются, что тоже не указывает на переобучение.

#### 2. Влияние L1-регуляризации на качество модели
Что касается L1-регуляризации, то она не помогла улучшить модель. R² упал до 0.26, что хуже, чем у исходной модели. Это может означать, что регуляризация не смогла компенсировать снижение качества, и, похоже, не устранила признаки переобучения.

#### 3. Эффект добавления полиномиальных признаков
Добавление полиномов значительно ухудшило качество модели. На тренировочных данных R² стал отрицательным (-1.2248), что говорит о том, что модель не объясняет дисперсию данных и делает предсказания хуже, чем просто среднее значение целевой переменной.
На тестовых данных результат ещё хуже R² = -1.7065. 

Это явное переобучение — модель слишком сложна для данных, добавленные полиномиальные признаки привели к сильному увеличению ошибки как на тренировочных, так и на тестовых данных. Модель слишком точно подогналась под тренировочные данные, что привело к сильному снижению её обобщающей способности на тестовых данных.

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

Когда ~~мы с чатом гпт~~ я работала над улучшением качества модели, наткнулась на интересный метод под названием **Voting Regressor**.  
(супер много вариантов перепробовала, этот единственный дал хоть что-то относительно существенное)

Как я поняла, **Voting Regressor** работает следующим образом: он обучает несколько моделей на одних и тех же данных, а затем комбинирует их предсказания. Например, в моей реализации я взяла простую линейную регрессию и случайный лес — два довольно разных подхода. Вместо того чтобы полагаться на одну из этих моделей, **Voting Regressor** усредняет их предсказания, что помогает снизить ошибку и улучшить обобщающую способность модели.

In [72]:
from sklearn.ensemble import VotingRegressor, RandomForestRegressor

voting_regressor = VotingRegressor(estimators=[
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor(n_estimators=100))
])
voting_regressor.fit(X_train, y_train)
y_pred_test_voting = voting_regressor.predict(X_test)

print('Voting Regressor R²:', r2_score(y_test, y_pred_test_voting))

Voting Regressor R²: 0.3765221267960679


## Часть 2. Target encoding (**всего 5 баллов**)

Перед проверкой!

Я часто пользовалась Chat GPT: просила его объяснить все моменты, много уточняющих вопросов задавала, но еще сильно помогли интрукции, которые до заданий. Спасибо за них!

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [2]:
sales = pd.read_csv('/Users/kh11vika/Downloads/sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [3]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`.

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [4]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621764


#### Способ 2

In [5]:
'''
     Differently to `.target.mean()` function `transform`
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621764


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection.

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях.

In [6]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=False)
item_target_enc = np.zeros(all_data.shape[0])

for train_index, test_index in kf.split(all_data):
    train_data = all_data.iloc[train_index]
    test_data = all_data.iloc[test_index]
    
    item_id_target_mean = train_data.groupby('item_id')['target'].mean()
    item_target_enc[test_index] = test_data['item_id'].map(item_id_target_mean)

all_data['item_target_enc'] = item_target_enc
all_data['item_target_enc'].fillna(0.3343, inplace=True)

correlation = np.corrcoef(all_data['target'].values, all_data['item_target_enc'].values)[0][1]
print(f'Корреляция: {correlation:.4f}')

Корреляция: 0.4165


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`.

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [9]:
total_target_sum = all_data.groupby('item_id')['target'].transform('sum')
n_objects = all_data.groupby('item_id')['target'].transform('size')

all_data['item_target_enc'] = (total_target_sum - all_data['target']) / (n_objects - 1)
all_data['item_target_enc'].fillna(0.3343, inplace=True)

zakodirovannaya = all_data['item_target_enc']
correlation = np.corrcoef(all_data['target'], zakodirovannaya)[0, 1]
print(f'Корреляция: {correlation:.4f}')

Корреляция: 0.4804


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конкретной категории, а не количество строк в датасете.

In [101]:
localmean = all_data.groupby('item_id').target.mean()  
nrows = all_data.groupby('item_id').target.count()

all_data['item_target_enc'] = all_data['item_id'].map((localmean * nrows + 0.3343 * 100) / (nrows + 100))

all_data['item_target_enc'].fillna(0.3343, inplace=True)  
zakodirovannaya = all_data['item_target_enc'].values

correlation = np.corrcoef(all_data['target'].values, zakodirovannaya)[0][1] 
print(f'Корреляция: {correlation:.4f}')

Корреляция: 0.4818


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [10]:
all_data = all_data.sort_values(['date_block_num', 'shop_id', 'item_id'])

cumulative_sum = all_data.groupby('item_id')['target'].cumsum()
previous_target_sum = cumulative_sum - all_data['target']
item_counts = all_data.groupby('item_id')['target'].cumcount()

all_data['item_target_enc'] = previous_target_sum / item_counts
all_data['item_target_enc'].fillna(0.3343, inplace=True)

zakodirovannaya = all_data['item_target_enc'].values  

correlation = np.corrcoef(all_data['target'].values, zakodirovannaya)[0][1] 
print(f'Корреляция: {correlation:.4f}')

Корреляция: 0.5025


Ожидаемый ответ 0.5025